# Predicting Player Goal Counts for the Maple Leafs
___

In [232]:
import psycopg2
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
from sklearn import svm, preprocessing, pipeline, set_config, compose, model_selection

### Data Preparation
___

In [193]:
# create connection to database
connection = psycopg2.connect(
    host="localhost",
    database="50_in_07",
    user="tanyatang")

In [195]:
# close connection
connection.close()

### Data Visualization
___

In [180]:
# set maplotlib parameters
plt.style.use('seaborn-darkgrid')
plt.rc('axes', labelsize=12)
plt.rc('xtick', labelsize=10)
plt.rc('ytick', labelsize=10)
plt.rc('legend', fontsize=10)
plt.rcParams['figure.figsize'] = (10, 6)

### Elastic-Net Regression
___

### Multi-Layer Perceptron Regression
___

### Results and Analysis
___

### Model Export
___